## Overview

To facilitate graph analysis, we need to support sharing layouts between graphs, something I haven't seen done very well elsewhere.  In particular, it should be possible to:

* Capture the layout from a graph $G_0$, either:
    * &#x2713; After creating the graph mark.
    * Without creating a graph mark.
* Use that layout to render a graph $G_1$.  This should work in all cases:
    * &#x2713; $G_1$ has the same vertices as $G_0$, in the same order.
    * $G_1$ has the same vertices as $G_0$, in a different order.
    * $G_1$ has a subset of the vertices in $G_0$, in arbitrary order.
    * $G_1$ has a superset of the vertices in $G_0$, in arbitrary order.
        * In this case the new vertices should be laid-out using the chosen layout algorithm, without disturbing the existing vertices.

To create a layout without creating a graph mark:

```python
layout = toyplot.layout.graph(edges) # Induce vertices from edges
layout = toyplot.layout.graph(edges, vcount) # In case there are unconnected vertices
layout = toyplot.layout.graph(edges, vcoordinates) # Specify vertex coordinates (including unconnected)
layout = toyplot.layout.graph(sources, targets) # Induce vertices from edges
layout = toyplot.layout.graph(sources, targets, vcount) # In case there are unconnected vertices
layout = toyplot.layout.graph(sources, targets, vcoordinates) # Specify vertex coordinates (including unconnected)
```

The returned layout object has a subset of the properties of toyplot.mark.Graph:

```python
layout.vcount
layout.vid
layout.vcoordinates
layout.ecount
layout.esource
layout.etarget
layout.eshape
layout.ecoordinates
layout.edges?
```

In [1]:
import numpy
import toyplot.color
import toyplot.generate

In [2]:
numpy.random.seed(1234)

In [7]:
# Random graph
vertices = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
random_graph = numpy.random.choice(vertices, (40, 2))

# Random tree
random_tree = toyplot.generate.prufer_tree(numpy.random.choice(4, 12))
#edges = toyplot.generate.prufer_tree([1,1])

reingold_a = numpy.array([[0,1],[0,2],[1,3],[1,4],[2,5],[2,6],[3,7],[3,8],[4,9],[4,10],[6,11],[9,12],[9,13],[10,14],[10,15],[12,16],[14,17]])
reingold_b = numpy.array([[0,1],[0,2],[0,3],[0,4],[0,5],[0,6],[1,7],[1,8],[1,9],[1,10],[1,11],[1,12],[1,13],[4,14],[6,15],[13,16],[15,17],[15,18],[15,19],[15,20],[15,21],[15,22],[15,23]])
reingold_c = numpy.array([[0,1],[0,2],[1,3],[2,4],[2,5],[3,6],[4,7],[5,8],[5,9],[6,10],[7,11],[9,12],[10,13],[12,14]])

# Ring graph
#source = numpy.arange(6)
#target = (source + 1) % 6
ring = numpy.column_stack((numpy.arange(6), (numpy.arange(6) + 1) % 6))
                          
graph = ring

In [8]:
%%time

colormap = toyplot.color.LinearMap(toyplot.color.Palette(["white", "yellow", "orange", "red"]))

canvas, axes, mark = toyplot.graph(
    graph,
    #layout=toyplot.layout.FruchtermanReingold(),
    #layout=toyplot.layout.Buchheim(),
    vcolor=colormap,
    vmarker="o",
    #varea=40,
    vsize=20,
    vopacity=1,
    vstyle={"stroke":"black"},
    ecolor="black",
    eopacity=0.2,
    estyle={},
    width=800,
    height=800,
);
axes.show = False
axes.padding = 10

vcoordinates = numpy.copy(mark.vcoordinates)
axes.text(vcoordinates.T[0], vcoordinates.T[1], mark.vid, color="black")

INFO:toyplot:Graph layout time: 34.4989299774 ms


CPU times: user 40.7 ms, sys: 2.33 ms, total: 43 ms
Wall time: 41.6 ms


In [9]:
vcoordinates[numpy.where(mark.vid == "b"), 0] += 1.0
canvas, axes, mark = toyplot.graph(graph, vcoordinates)

INFO:toyplot:Graph layout time: 0.0770092010498 ms
